In [1]:
import glob
import os
import numpy as np
from keras.models import load_model
import tensorflow as tf
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# Select 10 data files
ust_subfolder = input()
test_data = glob.glob('/beegfs/dr2915/sonyc_ust/db_mels/' + ust_subfolder + '/*.npz')

random_test_file_indices = np.random.randint(len(test_data), size = 10)

raw_audio = []
cmsis_mels = []
for index in random_test_file_indices:
    cmsis_mels.append(np.load(test_data[index])['db_mels'])
    raw_audio.append(np.load(os.path.join('/beegfs/dr2915/sonyc_ust/frames/8KHz',
                                          os.path.basename(test_data[index])))['audio'])

raw_audio = np.array(raw_audio)
print("raw_audio shape (before reshape): {}".format(raw_audio.shape))
raw_audio = raw_audio.reshape(-1, 8000)
print("raw_audio shape (after reshape): {}".format(raw_audio.shape))

test
raw_audio shape (before reshape): (10, 91, 8000)
raw_audio shape (after reshape): (910, 8000)


In [3]:
# Prepare input for Keras model
cmsis_mels = np.array(cmsis_mels).reshape(-1, 64, 51)[:,:,:,np.newaxis]

# Prepare input for TFLite model
*cmsis_mels_unstacked, = cmsis_mels[:,np.newaxis,:,:]

In [4]:
# Load Keras model
keras_model = load_model('/scratch/dr2915/Nathan/pipeline_cmsis_mels.h5')

# Keras softmaxes
keras_softmax = keras_model.predict(cmsis_mels)

/home/Roy.174/.local/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
# Load TFLite model
tflite_model = tf.lite.Interpreter(model_path="/scratch/sk7898/quantization/"
                                   "pipeline_cmsis_mels/tf_2_full_quantized_default_float32.tflite")
tflite_model.allocate_tensors()
input_details = tflite_model.get_input_details()
output_details = tflite_model.get_output_details()

# TFLite softmaxes
tflite_softmax = []
for mel in tqdm(cmsis_mels_unstacked):
    tflite_model.set_tensor(input_details[0]['index'], mel)
    tflite_model.invoke()
    tflite_softmax.append(tflite_model.get_tensor(output_details[0]['index']))
    
tflite_softmax = np.array(tflite_softmax).reshape(-1, 8)

100%|██████████| 910/910 [07:04<00:00,  2.15it/s]


In [6]:
# Save test data and softmaxes
np.savez_compressed('cmsis_mel_' + ust_subfolder, 
                    raw_audio=raw_audio, cmsis_db_mels=cmsis_mels, 
                    keras_softmax=keras_softmax, tflite_softmax=tflite_softmax)

In [7]:
# Load npz and get the number of compatible TFLite predictions
cmsis_mel_test = np.load('cmsis_mel_' + ust_subfolder + '.npz')

tflite_softmax_from_npz = cmsis_mel_test['tflite_softmax']
keras_softmax_from_npz = cmsis_mel_test['keras_softmax']

print("Number of TFLite predictions that match with Keras: {}/{}"
      .format(np.sum(np.argmax(tflite_softmax_from_npz, axis=-1)
                     ==np.argmax(keras_softmax_from_npz, axis=-1)), len(keras_softmax_from_npz)))

Number of TFLite predictions that match with Keras: 848/910
